# Step 3: Train and evaluate the Calorimetric method

In [1]:
distances = [15, 35]
doubleplanes = [8, 12, 20, 30]
energies = [200, 600, 1000]
erels = [100, 500, 1000, 2000, 3000]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["bert", "bic", "inclxx"]

## a) Train calorimetric method for all settings

In [2]:
import os
import wurlitzer
import joblib
from helpers import filename_for
import ROOT

Welcome to JupyROOT 6.16/00


In [3]:
ROOT.ROOT.EnableThreadSafety()
ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")
ROOT.gROOT.SetBatch(True)

In [4]:
def calibr(distance, doubleplane, energy, erel, nmax, physics):
    outfile = filename_for(
        distance, doubleplane, energy, erel, nmax, physics, ".ncut.root"
    )
    logfile = filename_for(
        distance, doubleplane, energy, erel, nmax, physics, ".ncut.log"
    )

    if os.path.exists(outfile):
        os.remove(outfile)

    # Write all output to a log file
    with open(logfile, "w") as log, wurlitzer.pipes(
        stdout=log, stderr=wurlitzer.STDOUT
    ):
        cal = ROOT.Neuland.Neutron2DCalibr(nmax)
        for neutron in range(1, nmax + 1):
            digifile = filename_for(
                distance, doubleplane, energy, erel, neutron, physics, ".digi.root"
            )
            cal.AddClusterFile(digifile)

        # Starting parameters (double val, double step, double lower, double upper)
        vslope = ROOT.std.vector("double")()
        vslope += [0.04, 0.001, 0.001, 10]
        vdistance = ROOT.std.vector("double")()
        vdistance += [energy / 20.0, energy / 100.0, energy / 100.0, energy]
        vdist_off = ROOT.std.vector("double")()
        vdist_off += [3, 0.5, 3, 6]

        # Create Cuts
        cal.Optimize(vslope, vdistance, vdist_off)

        # Write the output files (root, dat, pdf)
        cal.WriteParameterFile(outfile)
        o = ROOT.std.ofstream(outfile.replace(".root", ".dat"))
        cal.Print(o)
        cal.Draw(outfile.replace(".root", ".pdf"))
        del cal

In [5]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
    joblib.delayed(calibr)(
        distance=distance,
        doubleplane=doubleplane,
        energy=energy,
        erel=erel,
        nmax=neutron,
        physics=physics,
    )
    for distance in distances
    for energy in energies
    for doubleplane in doubleplanes
    for neutron in neutrons
    for erel in erels
    for physics in physicss
);

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1138 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 1688 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 13.8min finished


## b) Apply cuts to a setting and evaluate performance

In [5]:
import numpy as np
import pandas as pd

For each event, apply the cuts and list the predicted multiplicity along with the number of incoming neutrons, the number of primary points, etc.

In [6]:
def neuland_calorimetric_data(distance, doubleplane, energy, erel, nmax, physics):
    ncutfile = filename_for(
        distance, doubleplane, energy, erel, nmax, physics, ".ncut.root"
    )
    file = ROOT.TFile.Open(ncutfile)
    para = file.Get("R3BNeulandNeutron2DPar")

    data = []
    num_ex = 0
    for neutron in range(1, nmax + 1):
        digifile = filename_for(
            distance, doubleplane, energy, erel, neutron, physics, ".digi.root"
        )
        tfile = ROOT.TFile.Open(digifile)
        ttree = tfile.Get("evt")
        for event in ttree:
            # nPN: Number of incoming primary neutrons
            nIn = neutron
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            nPP = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            nPH = event.NeulandPrimaryHits.GetEntries()
            # nHits: Number of hits
            num_hits = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            num_clusters = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            try:
                edep = round(sum([hit.GetE() for hit in event.NeulandHits]))
            except:
                num_ex += 1
                edep = 0

            # Use calibrated calorimetric method (R3BNeulandNeutron2DPar) from the parameter file
            # to predict the multiplicity
            mult = para.GetNeutronMultiplicity(edep, num_clusters)

            data.append([nIn, nPP, nPH, num_hits, num_clusters, edep, mult])
    datapd = pd.DataFrame(data)
    datapd.columns = ["nIn", "nPP", "nPH", "num_hits", "num_cluster", "edep", "mult"]
    return datapd

In [7]:
distance = 15
doubleplane = 30
erel = 500
nmax = 5
physics = "inclxx"

data = {
    energy: neuland_calorimetric_data(
        distance, doubleplane, energy, erel, nmax, physics
    )
    for energy in [200, 600, 1000]
}

pd.options.display.max_rows = 20
display(data[600])

,nIn,nPP,nPH,num_hits,num_cluster,edep,mult
0,1,1,1,8,2,203,1
1,1,1,1,12,1,320,1
2,1,1,1,13,6,287,1
3,1,1,1,12,5,317,1
4,1,1,1,14,8,287,1
...,...,...,...,...,...,...,...
49995,5,5,5,29,17,693,3
49996,5,5,5,68,13,981,3
49997,5,4,4,46,25,771,3
49998,5,5,5,67,29,991,4


Annotate the image created by the calibration. Ready for use in the paper.

In [8]:
import shutil
import subprocess

energy = 600
ncutpdf = filename_for(distance, doubleplane, energy, erel, nmax, physics, ".ncut.pdf")
shutil.copyfile(ncutpdf, ncutpdf.replace(f"output/{physics}/", "paper/"))
subprocess.call(["lualatex", "calibr.tex"], cwd="paper", stdout=subprocess.DEVNULL)

0

Example neutron separation matrices and balanced accuracy score

In [9]:
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

out = []
for energy, df in data.items():
    print(energy)
    y_true = df[["nIn"]].values.ravel()
    y_pred = df[["mult"]].values.ravel()

    bac = balanced_accuracy_score(y_true, y_pred)
    print(bac)
    # cm = confusion_matrix(y_true, y_pred, labels=range(0, 6))
    # print(np.swapaxes(cm, 0, 1))
    cmrel = confusion_matrix(y_true, y_pred, labels=range(0, 6), normalize="true")
    cmrel = np.swapaxes(cmrel, 0, 1)
    print(cmrel)
    out.append([energy, bac, cmrel])

200
0.6133599999999999
[[0.    0.13  0.011 0.001 0.    0.   ]
 [0.    0.821 0.231 0.036 0.005 0.001]
 [0.    0.049 0.659 0.287 0.065 0.013]
 [0.    0.    0.099 0.559 0.322 0.099]
 [0.    0.    0.    0.115 0.484 0.344]
 [0.    0.    0.    0.001 0.124 0.544]]
600
0.62544
[[0.    0.069 0.005 0.    0.    0.   ]
 [0.    0.811 0.158 0.02  0.002 0.   ]
 [0.    0.12  0.727 0.263 0.055 0.009]
 [0.    0.    0.11  0.63  0.371 0.112]
 [0.    0.    0.    0.085 0.508 0.429]
 [0.    0.    0.    0.    0.064 0.45 ]]
1000
0.63916
[[0.    0.052 0.003 0.    0.    0.   ]
 [0.    0.781 0.133 0.014 0.001 0.   ]
 [0.    0.158 0.735 0.238 0.046 0.006]
 [0.    0.003 0.119 0.659 0.346 0.101]
 [0.    0.003 0.004 0.081 0.547 0.42 ]
 [0.    0.004 0.006 0.008 0.059 0.473]]


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Create a Latex file with the multiplicity matrices for use in the paper

In [10]:
def to_tabular(energy, bac, cm):
    text = (
        """\\resizebox{0.32\\textwidth}{!}{
\\begin{tabular}{cc|S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]}
\\toprule
\\multicolumn{2}{c|}{"""
        + str(energy)
        + """} & \\multicolumn{5}{c}{generated} \\\\
\\multicolumn{2}{c|}{"""
        + str("MeV")
        + """} & 1 & 2 & 3 & 4 & 5 \\\\
\\midrule
\\multirow{6}{*}{\\rotatebox[origin=c]{90}{detected}}
"""
    )
    for i, a in enumerate(cm[:nmax+1]):
        text += " & " + str(i)
        # text += " & ".join([ for x in a[1:]])
        for j, x in enumerate(a[1:nmax+1]):
            if j == i - 1:
                text += " & \\textbf{" + str(int(round(x * 100))) + "}"
            else:
                text += " & " + str(int(round(x * 100)))
        text += " \\\\\n"
    text += """\\bottomrule
\\end{tabular}}"""
    return text

In [11]:
def to_full_latex(data):
    text = """\\documentclass{{scrartcl}}
\\usepackage{booktabs}
\\usepackage{siunitx}
\\usepackage{multirow}
\\usepackage{graphicx}
\\begin{document}
\\begin{table}
"""
    text += f"""\\captionabove[Neutron separation matrices for multiplicities of 1 to {nmax} neutrons]{{
Neutron separation matrices for multiplicities of 1 to {nmax} neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the calorimetric neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND with {doubleplane} doubleplanes was located at a distance of {distance} m to the target.
Neutrons were generated with a relative energy of {erel} keV with respect to a medium heavy projectile fragment.
The distance between target and NeuLAND was filled with air and a 4\\,mm steel window.
Simulated with Geant4 using the \\texttt{{QGSP\\_{physics.upper()}\\_HP}} physics list.
Note that other maximum multiplicities will result in different efficiencies.
\\label{{tab:neutron-separation-matrices}}}}
"""
    for o in out:
        text += to_tabular(*o) + "\n"
    text += """\\end{table}
\\end{document}
"""
    return text

In [12]:
latex = to_full_latex(out)
print(latex)
with open(f"paper/mult{nmax}.tex", "w") as f:
    f.write(latex)
subprocess.call(["lualatex", f"mult{nmax}.tex"], cwd="paper", stdout=subprocess.DEVNULL)

\documentclass{{scrartcl}}
\usepackage{booktabs}
\usepackage{siunitx}
\usepackage{multirow}
\usepackage{graphicx}
\begin{document}
\begin{table}
\captionabove[Neutron separation matrices for multiplicities of 1 to 5 neutrons]{
Neutron separation matrices for multiplicities of 1 to 5 neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the calorimetric neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND with 30 doubleplanes was located at a distance of 15 m to the target.
Neutrons were generated with a relative energy of 500 keV with respect to a medium heavy projectile fragment.
The distance between target and NeuLAND was filled with air and a 4\,mm steel window.
Simulated with Geant4 using the \texttt{QGSP\_INCLXX\_HP} physics list.
Note that other maximum multiplicities will result in different efficiencies.
\label{tab:neutron-separation-

0